# Getting started with Data Quibbler

Below is a quick tutorial of *Quibbler* for *Python* (`pyquibbler`). It briefly shows some of the basic functionality and concepts, providing links to more compehensive descriptions on each topic.

You may also consult with the [[Examples]] to quickly demo some of the key functionalities of *Quibbler*.

## Setting up

### Install

To install *Quibbler* use: [Maor: anything else? specify the version requirements?]

`pip install pyquibbler`

### Import

`pyquibbler` is conventionally imported as `qb`. In addition, it is convenient to directly import some key functions such as `iquib` and `q` (which will be explained below). Following import, we need to also execute `qb.override_all()` which configure *NumPy* and *Matplotlib* functions to work with *Quibbler*. Imports of *NumPy* and *Matplotlib*, if needed, shoud follow the execution of `qb.override_all()`. [Maor: correct? Anything else?] A typical import therefore looks like:

In [11]:
# importing and initializing pyquibbler:
import pyquibbler as qb
from pyquibbler import iquib, q
qb.override_all()

# any other imports:
import matplotlib.pyplot as plt
import numpy as np

### Graphics backend

`pyquibbler` works well with `tk` and `widget` [Maor: anything else?].

If you are using *Jupiter Notebook*, use for example:

In [12]:
%matplotlib tk

## The quib object

A `Quib` is an object that represents an output *value* as well as the *function* and *arguments* used to calculate this value.  There are two major types of quibs: *input-quibs* (i-quibs) which take any *Python* object [Maor: any object - true?] as their argument and present it as their value (their function is trivially the identity function), and *function-quibs* (f-quibs) that produce their output value by applying a given function to a given list of arguments, which could be i-quibs, other f-quibs and any other *Python* objects. 

### Input quib (i-quib)

Any *Python* object can be transformed into an i-quib using the `iquib` command. For example:

In [13]:
my_lucky_number = iquib(7)
my_lucky_number

my_lucky_number = iquib(7)

Note that the string representation of a quib shows its *name* (in this case 'my_lucky_number'; see [[name]] property) and its *function* and *arguments* (in this case, 'iquib(7)'; See [[func]], [[args]], [[kwargs]] properties). 

#### Getting the quib's value using `get_value()`
To get the output *value* of the quib, we use the [[get_value]] method:

In [14]:
my_lucky_number.get_value()

7

#### Input quibs can represent objects of any class [Maor: true?]
Quibs can represent any *Python* object including Numeric, String, List, Tuple, Set, and
Dictionary. They can also represent *NumPy* ndarrays, *Matplotlib* Artists as well as essentially any other type of objects.

For example:

In [15]:
city_data = iquib({'City':'Haifa', 'Population':279247})
city_data

city_data = iquib({'City': 'Haifa', 'Population': 279247})

In [16]:
hello_world = iquib(['Hello','World'])
hello_world.get_value()

['Hello', 'World']

#### Assigning new values to input quibs

Input quibs can be modified by assignments using standard *Python* assignment syntax:

In [17]:
hello_world[0] = 'Hi'
hello_world.get_value()

['Hi', 'World']

To completely replace the value of a quib, even with objects of a different type, use the `assign_value()` method:

In [18]:
city_data.assign_value('anything')
city_data.get_value()

'anything'

### Function quib (f-quib)

#### Applying functions or operators to quib arguments creates a *function-quib* that performs these operations
Quibbler modifies standard functions and operators suich that can work directly with quib arguments, or with combinations of quibs and other objects, to create a new *f-quib*, whose function is to perform the indicated operation. When we execute `qb.override_all()` (see in the import abobe),  *Quibbler* modifies many standard *Python*, *NumPy* and *Matplotlib* functions such that can work directly with quib arguments. Such Quibbler-adapted functions are called _quiby functions_ (see [full list[Quiby functions]]). Operators, such as `+`, `-`, `<`, `>`, `**`, `@`, etc. are also quiby, and so are all types of indexing including slicing, field access, and advanced indexing [Maor: correct?]. We can therefore easily define a chanined network of functional quibs using standard *Python* syntax.

As a simple example, let's start with an input quib `z` representing a numeric *NumPy* array:

In [19]:
z = iquib(np.array([2, 1, 2, 3]))

We can use this quib in standard functions and operations, just like we would use a normal numeric *NumPy* array. For example:

In [20]:
z_sqr = z ** 2
z_sqr

z_sqr = z ** 2

The statement above created `z_sqr` which is a functional quib whose _function_ is to square the *value* of `z`.

We can similarly continue with additional downstream operations. Say, calculating the average of the elements of `z_sqr`:

In [21]:
mean_z_sqr = np.average(z_sqr)
mean_z_sqr

mean_z_sqr = average(z_sqr)

#### Quibs are defined declaratively (lazy evaluation) 

In general, quib operations are declarative; they define a quib with a specified function and arguments, but do not immediately execute this function. For example, the statement above, `mean_z_zqr = np.average(z_sqr)` creates a new quib whose function is to perform `np.average` on the value of `z_sqr`, but this averaging operation is not yet computed (deferred evaluation). Instead, as shown below, the quib's function is only evaluated when we ask for the value of the quib.

#### Quib functions are only evaluated when their output value is needed

To calculate the value of a funational-quib, we can use the `get_value()` method:

In [22]:
mean_z_sqr.get_value() # (2^2 + 1^2 + 2^2 + 3^2) / 4 = 4.5

4.5

The statement above triggers the evaluation of `mean_z_sqr`: performing the function `np.average` on the *value* of `z_sqr`. This operation, in turn, therefore also triggers the evaluation of `z_sqr`.

#### f-quibs cache their calculated value

Following calculation of its value, a quib can cache the result to avoid unnecessary future calculations. Repeated requests for the value of the quib thereby do not require recalculations. For more about caching, see the [[cache_behavior]] and [[cache_status]] properties. 

### Upstream changes automatically propagate to affect downstream results

When we make changes to a quib, these changes are automatically propagated downstream to affect the values of dependent quibs (recursively). For example, suppose we change one of the elements of our input quib `z`:

In [23]:
z[2] = 0

When such a change is made, downstream dependent quibs are notified that some or all of their cached output is no longer valid (though, no re-calculation is immediately being performed). Then, when we ask for the value of a downstream quib, it will get recalculated to reflect the upstream change:

In [24]:
mean_z_sqr.get_value() # (2^2 + 1^2 + 0^2 + 3^2) / 4 = 3.5

3.5

### Quib indexing too is interpreted decleratively

Similarily to applying functions on quib arguments, which returns a function quib, index a quib also creates a function quib whose function is to perform the indexing operation. 

For example, we can define a function quib that calculates the middle value of each two consecutive elements of an array:

In [25]:
r = iquib(np.array([0., 3., 2., 5., 8.]))
r_middle = (r[0:-1] + r[1:]) * 0.5
r_middle

r_middle = (r[0:-1] + r[1:]) * 0.5

In [26]:
r_middle.get_value()

array([1.5, 2.5, 3.5, 6.5])

Note that `r_middle` is defined functionaly; if its argument change it will get re-evaluated:

In [27]:
r[-1] = 13.
r_middle.get_value()

array([1.5, 2.5, 3.5, 9. ])

### Any function can be implemented as "_quiby_"

While many *Python*, *NumPy* and *Matplotlib* functions are supported to work directly on quibs [Quiby functions]], some functions are left regular, not quiby. In addition, any typical user function is generally not quiby. Therefore, to allow a generic way to create a function quib for _any_ function, even non-quiby, Quibbler provides a special syntax, called the _q-syntax_. Defining `w = q(fun, *args, **kwards)` creates a new function-quib `w` whose value is calculated by applyig the function `fun` on the provided args and kwards arguments, while replacing any quib arguments with their respective values. 

For example, to define a quib that implements a string-specific `format()` function (which is not a *Quibbler* function), we can use:

In [28]:
xy = iquib([2, 3])
xy_text = q('X={}, Y={}'.format, xy[0], xy[1])
xy_text.get_value()

'X=2, Y=3'

In [29]:
xy[1] = 5
xy_text.get_value()

'X=2, Y=5'

As another example, consider `str`: applied to quib, it returns its string represntation, rather than a new quib that performs `str` on the value of the argument quib. If we want the latter, we can use the `q` syntax:

In [30]:
w = iquib(7)
str_native = str(w)
str_native

'w = iquib(7)'

In [31]:
str_quiby = q(str, w)
str_quiby.get_value()

'7'

Other common *Python* functions that are not quiby, yet can be implemented using the `q` syntax include: `len`, `int`, `str` [Maor: others?]. 

### Calculation effeciency

As noted above, calculations in *Quibbler* are cached and are only repeated following changes to upstream inputs. Notably though, when upstream changes occur, *Quibbler* does not blindly invalidates all downstream results. Instead, it follows which are specific quibs and the specific pinpointed slices or elements thereof, that must be recalculated, thereby effeciently reducing required calculations. 

Consider the following example:

In [32]:
def my_mean(x):
    print('Calculating the mean of: ',x)
    return np.average(np.log(x))

In [33]:
v = iquib(np.array([1, 3, 2, 3, 1, 4]))
v_sqr = v ** 2
n = 3 #q(len, v) // 2
mean_v_sqr_left = q(my_mean, v_sqr[0:n]) # average of the first 3 elements of v_sqr
mean_v_sqr_right = q(my_mean, v_sqr[n:]) # average of the last 3 elements of v_sqr
mean_v_sqr = np.array([mean_v_sqr_left, mean_v_sqr_right])

In [34]:
mean_v_sqr.get_value()

Calculating the mean of:  [1 9 4]
Calculating the mean of:  [ 9  1 16]
Calculating the mean of:  [1 9 4]
Calculating the mean of:  [ 9  1 16]


array([1.19450631, 1.65660443])

In [35]:
v[5] = 5
mean_v_sqr.get_value() 
# Quibbler knows to only re-calculate the mean of the last 3-elements:

Calculating the mean of:  [ 9  1 25]


array([1.19450631, 1.8053668 ])

### Matplotlib functions too can work directly on quibs, creating "live" graphics

Graphics *Matplotlib* functions too can work directly with quib arguments, creating *graphics quibs*, which represent "live" graphics: graphics that automatically refreshes upon upstream changes. 

For example:

In [36]:
z = iquib(np.array([1., 2, 2, 3, 1, 4]))
z_sqr = z ** 2
mean_z_sqr = np.average(z_sqr)

In [37]:
plt.plot(z_sqr, '-o')
plt.plot([0, 5], mean_z_sqr + [0, 0],'k--')
plt.text(0, mean_z_sqr + 0.5, q('Average = {:.2f}'.format, mean_z_sqr))
plt.ylabel(str(z_sqr));
plt.ylim([0, 17]);

[[/images/graphics_refresh.gif]]

Note that unlike regular quibs which evaluate *lazily*, graphics quibs are evaluated *eagerly*, immediately upon creation, and are also recalculated immediately upon upstream changes, thereby enabling the above behavior.

### Quibbler readily creates interactive GUIs.

We have seen that graphics quibs automatically refresh when upstream changes occur, yet this data-to-graphics linkage can also be used reversely, readily creating interactive GUIs. Indicating `picer=True` for plots allows the user to drag the graphics and such changes in the graphics are inverted to upstream changes in the data (see also separate chapters on [[graphics]] and [[Inverse assignments]]).

For example, let's re-plot the data above, ploting both the input `z` and the output `z_aqr` and `mean_z_sqr` and indicating `picker=True`. As can be seen, the points can now be interactively dragged. Dragging `z` affect downstream results. More powerfully, even `z_sqr` can be dragged with these operations inverted to affect upstream `z` and downstream `mean_z_sqr`. 

In [38]:
plt.figure()
plt.subplot(2, 1, 1)
plt.plot(z, '-o', picker=True)
plt.ylabel('z');
plt.ylim([0, 5]);

In [39]:
plt.subplot(2, 1, 2)
plt.plot(z_sqr, '-o', picker=True)
plt.plot([0, 5], mean_z_sqr + [0, 0],'k--')
plt.text(0, mean_z_sqr + 0.5, q('Average = {:.2f}'.format, mean_z_sqr))
plt.ylabel(str(z_sqr));
plt.ylim([0, 17]);

[[imges/graphics_inverse.gif]]